In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import re
import nltk
from scipy import sparse
from nltk import bigrams    
from unicodedata import normalize
from nltk.corpus import stopwords
from collections import Counter
from IPython.display import Markdown

# Tema: Expansão de Consultas
### Autor: Luiz Fernando da Silva
Neste laboratório analizarei um conjunto de notícias do estadão armazenados em um arquivo csv utilizando técnicas de expansão de consultas para, por fim, responder as seguinstes questões:
* Quais os termos retornados para a expansão de cada consulta?
* Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
* Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
* A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?


In [2]:
dados = pd.read_csv('data/estadao_noticias_eleicao.csv')

# Função para limpar o texto
Essa Função remove todos os caracteres especiais do texto bem como sua acentuação.

In [3]:
def limpar_texto(texto):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', texto)

# Lista de stopwords
Criando lista de stopwords à ser removida da matrix de coocorrência

In [4]:
words = [limpar_texto(stopword) for stopword in stopwords.words('portuguese')]

# Join do conteúdo
Juntando os títulos das notícias com seus respectivos subtítulos e conteúdos, e também removendo das nóticias os caracteres especiais e a acentuação para posteriomente facilitar a tokenização.

In [5]:
conteudo = dados.titulo + " " + dados.subTitulo + " " + dados.conteudo
conteudo = conteudo.fillna("")
conteudo = conteudo.apply(limpar_texto)
ids = dados.idNoticia

# Tokenizando conteúdo
Criando tokens com cada palavra do texto para que posteriormente possam ser indexadas e associadas aos respectivos ids das notícias, e contando a frequência de cada termo no texto.

In [6]:
tokens = conteudo.apply(nltk.word_tokenize)
term_frequence = tokens.apply(Counter)

# Indexando tokens
Criando indices invertidos com os tokens para poder aplicar os métodos de busca.

In [7]:
index = {}

for i in range(len(tokens)):
    id_noticia = ids[i]
    palavras = tokens[i]
    for palavra in palavras:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = {}
        
        id_rec = index[palavra].get(id_noticia)
        
        if not id_rec:
            docs = index[palavra]
            docs[id_noticia] = term_frequence[i][palavra]

# Método que gera um dicionário com vetores de pesos
Este método gera uma dicionário cujas chaves são os ids documentos que contém os termos pesquisados e os valores são vetores que contem o peso de cada termo nos documentos, esse vetor é do tipo TF (Term Frequence).

In [8]:
def gera_docs_peso(termos):
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = index[termo]
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else tf for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                doc_vector[i] = tf
    return docs_peso


# Método que gera um vetor binário da consulta
Este método verifica se cada termo existe no indice e gera uma vetor binpario onde cada indice do vetor assume o valor de 0 se não existir ou 1 se existir.

In [9]:
def gera_query_vetor(termos):
    vetor = np.array([1 if index.get(termo) else 0 for termo in termos])
    return vetor

# Busca genérica
Este método encapsula as partes em comum de todos os métodos de busca implementados para evitar repetição de código.

In [10]:
def busca(termos, gerador_query, gerador_doc_vetor):
    docs_peso = gerador_doc_vetor(termos)
    query = gerador_query(termos)
    
    doc_rank = sorted(list(docs_peso.items()), key=lambda doc: np.dot(doc[1], query), reverse=True) 
    return [doc[0] for doc in doc_rank]

# Busca por term frequence (TF)
Busca vetorial pelo método TF.

In [11]:
def buscar_por_tf(termos):
    return busca(termos, gera_query_vetor, gera_docs_peso)

# Função para gerar matrix esparsa de coocorrência
Este código pode ser encontrado em: [https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb](https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb)

In [12]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

# Gerando tokens para criar a matrix esparsa
Neste trexo são criados sos tokens que serão utilizados no método co_ocurrence para criar matrix esparsa.

In [13]:
tokens_lists = conteudo.apply(lambda text: text.lower().split())

In [14]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list if token not in words]

# Criando matrix esparsa
Criando matrix esparsa para ser usada nas buscas expansivas

In [15]:
matrix, vocab = co_occurrence_matrix(tokens)

# Matrix de bigramas consultável

In [16]:
consultable_matrix = matrix.tocsr()

# Gera lista de coocorrência e lista de frequência
Essa função gera uma lista dos indices das palavras que mais coocorrem com o termo pesquisado ordenada em ordem decrescente pegando as primeiras 3 indeces de termos mais frequêntes e também gera uma lista com a respectiva frequência desses termos.

In [17]:
def get_co_ocurrence(word):
    list_of_occurency = consultable_matrix[vocab[word]].getrow(0).toarray()[0]
    indexs, frequency = zip(*sorted(enumerate(list_of_occurency), key=lambda x: x[1], reverse=True))
    return indexs[:3], frequency[:3]

# Metodo que faz uma busca expansiva
Usando o método get_co_ocurrence é gerada uma lista dos termos que mais coocorrem com o termo pesquisado, é feita uma busca utilizando os novos termos encontrados.

In [18]:
def busca_expansao(termo):
    ocurrecy = get_co_ocurrence(termo)
    expansao = [word for key in ocurrecy[0] for word in vocab.keys() if vocab[word] == key]
    expansao.append(termo)
    exp = buscar_por_tf(expansao)
    return (expansao, exp)
    

# Gerando Busca Expansiva
Aqui é gerada uma tabela contendo os resultados da busca expansiva e da busca original dos termos dilma, petrobras e aecio.

In [19]:
# Usando o termo Dilma
termo1 = 'dilma'
expansao1, doc_exp1 = busca_expansao(termo1)
busca_original1 = buscar_por_tf([termo1])

# Usando o termo petrobras
termo2 = 'petrobras'
expansao2, doc_exp2 = busca_expansao(termo2)
busca_original2 = set(buscar_por_tf([termo2]))

# Usanndo o termo aecio
termo3 = 'aecio'
expansao3, doc_exp3 = busca_expansao(termo3)
busca_original3 = set(buscar_por_tf([termo3]))

Markdown(
"""
**1. Tabela dos resultados das buscas originais e expandidas**

| Termo         | Termos expansão | Busca Original | Busca Expandida   |
|:-------------:|:---------------:|:--------------:|:-----------------:|
| {first_term}  | {first_exp}     | {first_ori}    | {first_srch_exp}  |
| {second_term} | {second_exp}    | {second_ori}   | {second_srch_exp} |
| {third_term}  | {third_exp}     | {third_ori}    | {third_srch_exp}  |

""".format(
    first_term=termo1,
    second_term=termo2,
    third_term=termo3,
    first_exp=', '.join(expansao1[:3]),
    second_exp=', '.join(expansao2[:3]),
    third_exp=', '.join(expansao3[:3]),
    first_ori=len(busca_original1),
    second_ori=len(busca_original2),
    third_ori=len(busca_original3),
    first_srch_exp=len(doc_exp1),
    second_srch_exp=len(doc_exp2),
    third_srch_exp=len(doc_exp3),
))


**1. Tabela dos resultados das buscas originais e expandidas**

| Termo         | Termos expanão | Busca Original | Busca Expandida   |
|:-------------:|:--------------:|:--------------:|:-----------------:|
| dilma  | rousseff, disse, aecio    | 2669    | 3853  |
| petrobras | paulo, graca, disse   | 963   | 3886 |
| aecio  | neves, disse, afirmou    | 1578    | 3658  |



# Análise dos resultados

* Quais os termos retornados para a expansão de cada consulta?
> Observando a tabela de resultados acima temos que os termos retornados para as buscas de Dilma, Petrobrás e Aécio são:
> * Dilma
>> * rousseff 
>> * disse 
>> * aecio
> * Petrobrás
>> * paulo 
>> * graca
>> * disse
> * Aécio
>> * neves
>> * disse
>> * afirmou


* Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
> <p style="text-align: justify">Os termos retornados estão relacionados com as consultas pois, são eles quem mais aparecem em conjunto com a busca desejada e podem se gerar uma maior aproximação com que o usuário deseja pesquisar. Podemo ver com mais clareza na tabela de resultados acima, onde ao pesquisar por dilma um dos resultados dos termos de expansão é Rousseff, que é seu sobrenome. Outra análise que pode ser feita é calcular quantos dos documentos retornados pela busca original também são retornados pela busca expandida. Neste caso, mesmo não adicionado os termos da consulta original e deixando apenas os termos que mais estão correlacionados para fazer a busca, a quantidade de documentos que estão na busca original e que não estão na busca expandida é mínima, o que mostra que esses termos estão bastante relacionados aos termos da consulta original.</p>

* Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
> <p style="text-align: justify">A busca expandida retorna resultados melhores para a busca, pois como ela usa os termos que estão mais relacionados com os termos originais, fica mais expressiva e retornará também resultados de outras possíveis palavras chave que o usuário poderia usar em futuras buscas. Entretanto, quanto mais termos a busca expandida tiver, maior vai ser a quantidade de documentos retornados por ela, e se, não houver um rankeamento adequado dos resultados, o usuário pode levar mais tempo procurando os documentos que realmente são relevantes para ele.</p>

* A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?
> <p style="text-align: justify">Expandindo a consulta o recall irá melhorar, pois ele é a razão entre o total de documentos relevantes na busca e e a quantidade total de documentos relevantes, ou seja, quanto maior for a quantidade de documentos retornados maior será a quantidade de documentos relevantes que estarão contidos nessa busca. Por outro lado, a precisão diminui já que ela é a razão entre a quantidade de documentos relevantes na busca e a quantidade de documentos totais presentes na busca. Formula do recall: $\frac{tp}{tp+fn}$; formula do precision: $\frac{tp}{tp+fp}$; onde, tp é o total de documentos relevantes na busca, fn é o total de documentos relevantes não presentes na busca e fp é o total de documentos não relevantes na busca.</p>